# Wide and Deep model

In [2]:
import sys
import pandas as pd
import numpy as np

import tensorflow as tf
import sklearn

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
sns.set_style("darkgrid", {"axes.facecolor": ".95"})
sns.set_context("paper")

In [3]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Model 1

<img src="images/wide-and-deep-01.png" width="400px">

In [5]:
input_A = tf.keras.layers.Input(shape=[5], name="wide_input")
input_B = tf.keras.layers.Input(shape=[6], name="deep_input")
hidden1 = tf.keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([input_A, hidden2])
output = tf.keras.layers.Dense(1, name="output")(concat)
model = tf.keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [12]:
model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_test_A, X_test_B))

Epoch 1/20
363/363 [==============================] - 0s 888us/step - loss: 0.4312 - val_loss: 0.4177
Epoch 2/20
363/363 [==============================] - 0s 718us/step - loss: 0.4289 - val_loss: 0.4109
Epoch 3/20
363/363 [==============================] - 0s 722us/step - loss: 0.4264 - val_loss: 0.4190
Epoch 4/20
363/363 [==============================] - 0s 725us/step - loss: 0.4241 - val_loss: 0.4123
Epoch 5/20
363/363 [==============================] - 0s 737us/step - loss: 0.4222 - val_loss: 0.4104
Epoch 6/20
363/363 [==============================] - 0s 743us/step - loss: 0.4203 - val_loss: 0.4157
Epoch 7/20
363/363 [==============================] - 0s 746us/step - loss: 0.4179 - val_loss: 0.4139
Epoch 8/20
363/363 [==============================] - 0s 740us/step - loss: 0.4165 - val_loss: 0.3995
Epoch 9/20
363/363 [==============================] - 0s 747us/step - loss: 0.4149 - val_loss: 0.3970
Epoch 10/20
363/363 [==============================] - 0s 730us/step - loss: 0.413

In [14]:
from sklearn.metrics import mean_absolute_percentage_error

mean_absolute_percentage_error(y_test, y_pred)

0.2687186708204187

----

# Model 3

<img src="images/wide-and-deep-03.png" width="400px">

In [15]:
input_A = tf.keras.layers.Input(shape=[5], name="wide_input")
input_B = tf.keras.layers.Input(shape=[6], name="deep_input")
hidden1 = tf.keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([input_A, hidden2])
output = tf.keras.layers.Dense(1, name="main_output")(concat)
aux_output = tf.keras.layers.Dense(1, name="aux_output")(hidden2)
model = tf.keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])

In [17]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3))

In [18]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20,
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Epoch 1/20

363/363 [==============================] - 1s 1ms/step - loss: 2.2619 - main_output_loss: 2.1355 - aux_output_loss: 3.3996 - val_loss: 1.3348 - val_main_output_loss: 1.1052 - val_aux_output_loss: 3.4019
Epoch 2/20
363/363 [==============================] - 0s 864us/step - loss: 0.9261 - main_output_loss: 0.8068 - aux_output_loss: 1.9991 - val_loss: 0.9992 - val_main_output_loss: 0.7426 - val_aux_output_loss: 3.3086
Epoch 3/20
363/363 [==============================] - 0s 796us/step - loss: 0.7717 - main_output_loss: 0.6902 - aux_output_loss: 1.5047 - val_loss: 0.9032 - val_main_output_loss: 0.6669 - val_aux_output_loss: 3.0303
Epoch 4/20
363/363 [==============================] - 0s 824us/step - loss: 0.6880 - main_output_loss: 0.6228 - aux_output_loss: 1.2743 - val_loss: 0.8412 - val_main_output_loss: 0.6418 - val_aux_output_loss: 2.6358
Epoch 5/20
363/363 [==============================] - 0s 795us/step - loss: 0.6334 - main_output_loss: 0.5760 - aux_output_loss: 1.1499 -

In [19]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

1/1 [==============================] - 0s 35ms/step


In [21]:
y_pred_aux

array([[0.9203347],
       [1.842216 ],
       [2.6852145]], dtype=float32)

# Using subclassing API

In [22]:
class WideAndDeepModel(tf.keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel(30, activation="relu")

In [23]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=10,
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

Epoch 1/10
363/363 [==============================] - 1s 1ms/step - loss: 2.5452 - output_1_loss: 2.1606 - output_2_loss: 6.0065 - val_loss: 1.3578 - val_output_1_loss: 1.0430 - val_output_2_loss: 4.1907
Epoch 2/10
363/363 [==============================] - 0s 785us/step - loss: 1.0471 - output_1_loss: 0.7885 - output_2_loss: 3.3745 - val_loss: 0.9518 - val_output_1_loss: 0.7316 - val_output_2_loss: 2.9333
Epoch 3/10
363/363 [==============================] - 0s 820us/step - loss: 0.8352 - output_1_loss: 0.6816 - output_2_loss: 2.2175 - val_loss: 0.8550 - val_output_1_loss: 0.6557 - val_output_2_loss: 2.6485
Epoch 4/10
363/363 [==============================] - 0s 808us/step - loss: 0.7465 - output_1_loss: 0.6402 - output_2_loss: 1.7031 - val_loss: 0.8186 - val_output_1_loss: 0.6199 - val_output_2_loss: 2.6073
Epoch 5/10
363/363 [==============================] - 0s 787us/step - loss: 0.6980 - output_1_loss: 0.6133 - output_2_loss: 1.4605 - val_loss: 0.7931 - val_output_1_loss: 0.5962 

# Saving and Restoring

In [24]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, activation="relu", input_shape=[8]),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(1)
])    

model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)

Epoch 1/10
363/363 [==============================] - 0s 889us/step - loss: 2.1707 - val_loss: 1.3049
Epoch 2/10
363/363 [==============================] - 0s 696us/step - loss: 0.7646 - val_loss: 0.6762
Epoch 3/10
363/363 [==============================] - 0s 716us/step - loss: 0.6873 - val_loss: 0.6537
Epoch 4/10
363/363 [==============================] - 0s 693us/step - loss: 0.6576 - val_loss: 0.6021
Epoch 5/10
363/363 [==============================] - 0s 695us/step - loss: 0.6319 - val_loss: 0.5801
Epoch 6/10
363/363 [==============================] - 0s 700us/step - loss: 0.6087 - val_loss: 0.5668
Epoch 7/10
363/363 [==============================] - 0s 707us/step - loss: 0.5873 - val_loss: 0.5526
Epoch 8/10
363/363 [==============================] - 0s 698us/step - loss: 0.5667 - val_loss: 0.5414
Epoch 9/10
363/363 [==============================] - 0s 696us/step - loss: 0.5474 - val_loss: 0.5055
Epoch 10/10
162/162 [==============================] - 0s 456us/step - loss: 0.502

**Saving**

In [25]:
model.save("my_keras_model.h5")

**Loading**

In [26]:
model = tf.keras.models.load_model("my_keras_model.h5")

**Predicting**

In [27]:
model.predict(X_test)

162/162 [==============================] - 0s 473us/step


array([[1.0178612],
       [1.288365 ],
       [2.7351105],
       ...,
       [1.6858722],
       [2.3802881],
       [3.4826055]], dtype=float32)

In [28]:
model.save_weights("my_keras_weights.ckpt")

In [29]:
model.load_weights("my_keras_weights.ckpt")

## Using Callbacks during Training

In [30]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, activation="relu", input_shape=[8]),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(1)
])    

In [32]:
class PrintValTrainRatioCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

In [35]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
val_train_ratio_cb = PrintValTrainRatioCallback()

model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3))

history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb, val_train_ratio_cb])

mse_test = model.evaluate(X_test, y_test)

Epoch 1/100
279/363 [======================>.......] - ETA: 0s - loss: 0.3454
val/train: 1.07
363/363 [==============================] - 0s 850us/step - loss: 0.3400 - val_loss: 0.3635
Epoch 2/100
292/363 [=======================>......] - ETA: 0s - loss: 0.3384
val/train: 1.11
363/363 [==============================] - 0s 718us/step - loss: 0.3395 - val_loss: 0.3759
Epoch 3/100
344/363 [===========================>..] - ETA: 0s - loss: 0.3420
val/train: 1.09
363/363 [==============================] - 0s 791us/step - loss: 0.3393 - val_loss: 0.3699
Epoch 4/100
272/363 [=====================>........] - ETA: 0s - loss: 0.3428
val/train: 1.04
363/363 [==============================] - 0s 779us/step - loss: 0.3390 - val_loss: 0.3510
Epoch 5/100
306/363 [========================>.....] - ETA: 0s - loss: 0.3330
val/train: 1.03
363/363 [==============================] - 0s 708us/step - loss: 0.3385 - val_loss: 0.3476
Epoch 6/100
293/363 [=======================>......] - ETA: 0s - loss: 0.34

---